## Orchestrate model evaluation with Amazon SageMaker Pipelines
  
The goal of this notebook is to provide an implementation of a multi-step SageMaker pipeline that will take care of multiple models evaluation, selection and registration into the SageMaker model registry.  
For running this example we will use **LLama-2-7b** models that will be used with default weights or after a finetuning. All the models will be instantiated and finetuned by using [Amazon Sagemaker Jumpstart SDK](https://aws.amazon.com/sagemaker/jumpstart/).  
This notebook is also using other Amazon SageMaker components:  

[SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/) is a purpose-built workflow orchestration service to automate all phases of machine learning (ML) from data pre-processing to model monitoring. With an intuitive UI and Python SDK you can manage repeatable end-to-end ML pipelines at scale. The native integration with multiple AWS services allows you to customize the ML lifecycle based on your MLOps requirements.
SageMaker Model Registry

[Amazon SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html) is a purpose-built metadata store to manage the entire lifecycle of ML models from training to inference. Whether you prefer to store your model artifacts (model framework files, container image) in AWS (Amazon ECR) or outside of AWS in any third party Docker repository, you can now track them all in Amazon SageMaker Model Registry. You also have the flexibility to register a model without read/write permissions to the associated container image. If you want to track an ML model in a private repository, set the optional ‘SkipModelValidation’ parameter to ‘All’ at the time of registration. Later you can also deploy these models for inference in Amazon SageMaker.

[Amazon SageMaker Clarify](https://aws.amazon.com/sagemaker/clarify/) provides purpose-built tools to gain greater insights into your ML models and data, based on metrics such as accuracy, robustness, toxicity, and bias to improve model quality and support responsible AI initiative. With the rise of generative AI, data scientists and ML engineers can leverage publicly available foundation models (FMs) to accelerate speed-to-market. To remove the heavy lifting of evaluating and selecting the right FM for your use case, Amazon SageMaker Clarify supports FM evaluation to help you quickly evaluate, compare, and select the best FM for your use case based on a variety of criteria across different tasks within minutes. It allows you to adopt FMs faster and with confidence.
To perform evaluation we are using the open source library [FMEval](https://github.com/aws/fmeval) that empowers SageMaker Clarify FM model evaluation.

This example is built by following the best practices explained in the blog post [Operationalize LLM Evaluation at Scale using Amazon SageMaker Clarify and MLOps services](https://aws.amazon.com/blogs/machine-learning/operationalize-llm-evaluation-at-scale-using-amazon-sagemaker-clarify-and-mlops-services/). 

### Environment setup
First we need to install required dependencies and import required libraries.  
We also make sagemaker SDK aware of the configuration file *config.yml*. 
This file *config.yml* contains general pipeline parameters like the default pipeline container instance type and the path to the file *dependencies.txt* with the required dependencies.
These dependencies will be automatically downloaded from the pipeline container at the start of each pipeline step. We will create *requirements.txt* file later in the notebook.

In [29]:
!pip3 install fmeval==0.4.0
!pip3 install sagemaker==2.208

In [30]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.function_step import step
from steps.jumpstart_deploy import jumpstart_deploy
from steps.jumpstart_finetune import jumpstart_finetune
from steps.selection import selection
from steps.preprocess import preprocess_chat
from steps.evaluation import evaluation
from steps.register import register
from steps.cleanup import cleanup
from steps.utils import create_training_job_name
import os

os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

### Evaluation and Fine-Tuning dataset preparation - preprocess step
*output_data_path* will contain the outputs artifacts of the pipeline.
We then add **preprocess** as our first pipeline step. This step will take care of any data preprocessing that must be done to create the evaluation and fine-tuning dataset.
In this example we are going to download the [SCIQ](https://huggingface.co/datasets/sciq) dataset and create from it two dataset for domain adaptation fine-tuning. We also create the evaluation dataset.
The output path of the datasets will be saved into *preprocess_step_ret* object.
Keep in mind the *pipeline_name* as it will be used in SageMaker Studio to find our pipeline in the UI.
Also keep in mind the path of the S3 bucket used as output for reviewing the output artifacts at the end.

In [31]:
pipeline_name = "mlops-model-finetune-evaluate-dft"

default_bucket = sagemaker.Session().default_bucket()
output_data_path = ("s3://"+ default_bucket + "/pipelines_examples/" + pipeline_name)

# You can add your own evaluation dataset code into this step
preprocess_step_ret = step(preprocess_chat, name="preprocess")(output_data_path)

print("The pipeline name is "+pipeline_name)
# Mark the name of this bucket for reviewing the artifacts generated by this pipeline at the end of the execution
print("Output S3 bucket: "+output_data_path)

The pipeline name is mlops-model-finetune-evaluate-dft
Output S3 bucket: s3://sagemaker-eu-west-1-944771376927/pipelines_examples/mlops-model-finetune-evaluate-dft


### Setup the models from SageMaker Jumpstart to be finetuned with a chat dataset
Setup the model_id from SageMaker Jumpstart that we are going to finetune with a chat dataset. We will use the default Jumpstart training parameters. The second model will be used as a baseline with default weights.

In [32]:
# We setup required model parameters
model_1 = {
    "model_id": "meta-textgeneration-llama-2-7b"
}
model_1["model_name"] = model_1["model_id"]+"_dft"

model_2 = {
    "model_id": "meta-textgeneration-llama-2-7b",
}
model_2["model_name"] = model_2["model_id"]+"_base"

model_list = [model_1, model_2]
evaluation_results_ret_list = []
deploy_ret_list = []

We are now going to create the pipeline steps. We start with a **finetune** step followed by a **deploy** and **evaluation** steps.
The result of evaluation will be saved into *evaluate_ret* object and appended into evaluation_results_ret_list to be used later.

In [33]:
finetune_ret = step(jumpstart_finetune, name="finetune_"+model_1["model_name"])(model_1, preprocess_step_ret)

# Deploy step is using the output from the finetune step (the training job name)
deploy_ret = step(jumpstart_deploy,
                                  name="deploy_"+model_1["model_name"])(model_1, finetune_ret)

# Evaluation step is using the output from preprocess (the S3 location of the evaluation dataset file) 
# and the output of the deploy step (the endpoint name)
evaluate_ret = step(evaluation,
                    name="evaluate_"+model_1["model_name"],
                    keep_alive_period_in_seconds=1200,
                    )(model_1,
                      preprocess_step_ret,
                      deploy_ret)

evaluation_results_ret_list.append(evaluate_ret)
deploy_ret_list.append(deploy_ret)

We create the same for the second model, deployed with default weights. We setup a **deploy** step followed by an **evaluation** step.
Like before, the result of evaluation will be saved into *evaluate_ret* object and appended into evaluation_results_ret_list to be used later.

In [34]:
deploy_ret = step(jumpstart_deploy, name="deploy_"+model_2["model_name"])(model_2)

# Evaluation step is using the output from preprocess (the S3 location of the evaluation dataset file) 
# and the output of the deploy step (the endpoint name)
evaluate_ret = step(evaluation,
                    name="evaluate_"+model_2["model_name"],
                    keep_alive_period_in_seconds=1200,
                    )(model_2,
                      preprocess_step_ret,
                      deploy_ret)

evaluation_results_ret_list.append(evaluate_ret)
deploy_ret_list.append(deploy_ret)

### Select best model and register it into SageMaker Model Registry
Now it's time to select best model. To do so we create a pipeline step dedicated to the best model **selection**.
The selection is using the output of all the models' evaluation.
The output of the **selection** step is the best model name. We will use the best model name in the **register** step.  
The **register** step will also need a package group and description name.

In [35]:
# Evaluation step is using the output from the evaluation steps of all the models
selection_ret = step(selection, name="best_model_selection")(*evaluation_results_ret_list)

# Set a package group name and description
model_package_group_name = "PipelineExampleFinetuneAndCompare"
model_package_group_description = "Pipeline Example Finetune And Compare"

# We will register the best model in the model register. The best model name is contained in the return object of the selection step
register_ret = step(register, name="register_model")(selection_ret,
                                                     deploy_ret,
                                                     output_data_path,
                                                     model_package_group_name,
                                                     model_package_group_description,
                                                     selection_ret)

### Cleanup
The last pipeline step is dedicated to cleanup all the resource that we are going to instantiate with the pipeline inside a **cleanup** step

In [36]:
# We append register_ret to make cleanup step dependent on register step
cleanup_ret = step(cleanup, name="cleanup")(register_ret, *deploy_ret_list)

### Creating and launching the pipeline
We are finally ready to create and launch the pipeline but before doing that we will need to create a requirements.txt file.
As a best practice we are reading the current sagemaker library version that we are using to create the pipeline and set it as a requirement into the requirement file.
Keeping the same sagemaker version in the creation and running phase will allow us to avoid any deserialization issues.

In [37]:
if os.path.exists("requirements.txt"):
    os.remove("requirements.txt")

with open('requirements.txt', 'w') as req_file:
    req_file.write("fmeval==0.4.0\n")
    req_file.write("sagemaker==" + str(sagemaker.__version__) + "\n")
    req_file.write("datasets\n")

In the last cell of this notebook we are creating the pipeline and serializing it to S3. 
Don't forget to attach the execution role with sufficient permission and the return results from the last steps of our pipeline.
We are now ready to start the pipeline execution!

In [38]:
from sagemaker import get_execution_role
role = get_execution_role()

pipeline = Pipeline(name=pipeline_name, steps=[cleanup_ret])#, sagemaker_session=session)
pipeline.upsert()
pipeline.start()

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.RoleArn


2024-06-20 08:10:39,078 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/cleanup/2024-06-20-08-10-27-958/function
2024-06-20 08:10:40,108 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/cleanup/2024-06-20-08-10-27-958/arguments
2024-06-20 08:10:47,960 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/var/folders/7j/mzj4jq4x6qlc63s7c977g8pc0000gs/T/tmp1kad9j_m/requirements.txt'
2024-06-20 08:10:48,409 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/cleanup/2024-06-20-08-10-27-958/pre_exec_script_and_dependencies'
2024-06-20 08:10:48,433 sagemaker.remote_function INFO     Copied user workspace to '/var/folders/7j/mzj4jq4x6qlc63s7c977

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.RoleArn


2024-06-20 08:10:52,361 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/register_model/2024-06-20-08-10-27-958/function
2024-06-20 08:10:53,091 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/register_model/2024-06-20-08-10-27-958/arguments
2024-06-20 08:10:54,478 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/var/folders/7j/mzj4jq4x6qlc63s7c977g8pc0000gs/T/tmpyosdbe_k/requirements.txt'
2024-06-20 08:10:54,947 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/register_model/2024-06-20-08-10-27-958/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.RoleArn


...

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.RoleArn


2024-06-20 08:11:19,062 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/evaluate_llama-2-7b/2024-06-20-08-10-27-958/function
2024-06-20 08:11:19,540 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/evaluate_llama-2-7b/2024-06-20-08-10-27-958/arguments
2024-06-20 08:11:19,959 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/var/folders/7j/mzj4jq4x6qlc63s7c977g8pc0000gs/T/tmpjnddn6_h/requirements.txt'
2024-06-20 08:11:20,146 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/evaluate_llama-2-7b/2024-06-20-08-10-27-958/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.RoleArn


2024-06-20 08:11:22,723 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/preprocess/2024-06-20-08-10-27-958/function
2024-06-20 08:11:23,202 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/preprocess/2024-06-20-08-10-27-958/arguments
2024-06-20 08:11:23,522 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/var/folders/7j/mzj4jq4x6qlc63s7c977g8pc0000gs/T/tmprfezwz_j/requirements.txt'
2024-06-20 08:11:23,843 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-eu-west-1-944771376927/mlops-model-finetune-evaluate-dft/preprocess/2024-06-20-08-10-27-958/pre_exec_script_and_dependencies'
2024-06-20 08:11:24,955 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-

_PipelineExecution(arn='arn:aws:sagemaker:eu-west-1:944771376927:pipeline/mlops-model-finetune-evaluate-dft/execution/1brxhphjzdse', sagemaker_session=<sagemaker.session.Session object at 0x7fab2a5cf880>)